In [1]:
from keras_unet.models import custom_unet
from keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
import tensorflow as tf 
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

IMG_LOC = '/home/clint/projects/DSP/data/p3/project3/data/'
MASK_LOC = '/home/clint/projects/DSP/data/p3/project3/masks/'
TRAIN_TXT = '/home/clint/projects/DSP/data/p3/project3/train.txt'
TEST_TXT = '/home/clint/projects/DSP/data/p3/project3/test.txt'

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [11]:
def img_preprocessing(img_loc, mask_loc):
    pairs = []
    with open(TRAIN_TXT) as train_file:
        for line in train_file:
            line = line.replace('\n', '')
            for im_file in os.listdir(f'{img_loc}/{line}/data/{line}'):
                pairs.append([f'{img_loc}/{line}/data/{line}/{im_file}',f'{mask_loc}/{line}.png'])

        return pairs
pairs = img_preprocessing(IMG_LOC, MASK_LOC)

In [16]:
train_pairs = pairs[:4000]
val_pairs = pairs[:4000]

In [8]:
from tensorflow.keras.utils import to_categorical ,Sequence
class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, pair, batch_size=16, dim=(224,224,3), shuffle=True):
        'Initialization'
        self.dim = dim
        self.pair = pair
#         self.class_map = class_map
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.pair) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.pair))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
            # Store sample
            img = Image.open(self.pair[i][0])
            mask = Image.open(self.pair[i][1])
            width, height = img.size   # Get dimensions
            left = (width - 256)/2
            top = (height - 256)/2
            right = (width + 256)/2
            bottom = (height + 256)/2
            # Crop the center of the image
            img = img.crop((left, top, right, bottom))
            mask = mask.crop((left, top, right, bottom))
            img = np.asarray(img, dtype=np.float32)/255
            batch_imgs.append(img)
            
            mask = np.asarray(mask, dtype=np.float32)/2
#             mask = form_2D_label(mask,self.class_map)
#             mask = to_categorical(mask , num_classes = 3)
            batch_labels.append(mask)
            
#             img = load_img(self.pair[i][0] ,target_size=self.dim)
#             img = img_to_array(img)/255.
#             batch_imgs.append(img)

#             label = load_img(self.pair[i][1],target_size=self.dim)
#             label = img_to_array(label)
#             label = form_2D_label(label,self.class_map)
#             label = to_categorical(label , num_classes = 32)
#             batch_labels.append(label)
            
        return np.array(batch_imgs) ,np.array(batch_labels)

In [17]:
img_size = 256
train_generator = DataGenerator(train_pairs,batch_size=4, dim=(img_size,img_size,1) ,shuffle=True)
train_steps = train_generator.__len__()
train_steps

1000

In [18]:
X,y = train_generator.__getitem__(1)
y.shape

(4, 256, 256)

In [19]:
val_generator = DataGenerator(val_pairs,batch_size=4, dim=(img_size,img_size,1) ,shuffle=True)
val_steps = val_generator.__len__()
val_steps

1000

In [29]:
def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def deconv_block(tensor, residual, nfilters, size=3, padding='same', strides=(2, 2)):
    y = Conv2DTranspose(nfilters, kernel_size=(size, size), strides=strides, padding=padding)(tensor)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y


def Unet(h, w, filters):
# down
    input_layer = Input(shape=(h, w, 1), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
    output_layer = Conv2D(filters=32, kernel_size=(1, 1), activation='softmax')(deconv9)

    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    return model

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical ,Sequence
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adadelta, Nadam ,Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard

Model: "Unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 256, 256, 64) 640         image_input[0][0]                
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 256, 256, 64) 256         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_36 (Activation)      (None, 256, 256, 64) 0           batch_normalization_36[0][0]     
_______________________________________________________________________________________________

In [30]:
model = Unet(img_size , img_size , 32)
model.summary()

Model: "Unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 256, 32) 320         image_input[0][0]                
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 256, 256, 32) 128         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_54 (Activation)      (None, 256, 256, 32) 0           batch_normalization_54[0][0]     
_______________________________________________________________________________________________

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy' ,metrics=['accuracy'])

In [32]:
mc = ModelCheckpoint(mode='max', filepath='top-weights.h5', monitor='val_acc',save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='val_acc', patience=10, verbose=0)
tb = TensorBoard(log_dir="logs/", histogram_freq=0, write_graph=True, write_images=False)
rl = ReduceLROnPlateau(monitor='val_acc',factor=0.1,patience=5,verbose=1,mode="max",min_lr=0.0001)
cv = CSVLogger("logs/log.csv" , append=True , separator=',')

In [ ]:
results = model.fit_generator(train_generator, 
                              steps_per_epoch=train_steps,
                              epochs=30,
                              validation_data=val_generator,
                              validation_steps=val_steps,
                              callbacks=[mc,es,tb,rl,cv])

Epoch 1/30
  23/1000 [..............................] - ETA: 30:23 - loss: 444.0157 - accuracy: 0.1096

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

import cv2